In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/scratch/cvlab/home/gwizdala/code/pidnet")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import torch

from functools import reduce
from pathlib import Path

from powerlines.utils import load_yaml

folds = load_yaml(Path("configs/powerlines/folds.yaml"))
timestamps = set(reduce(lambda a, b: a + b, folds.values(), []))

labels_folder = Path("/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/labels")
poles_distance_masks_folder = Path(
    "/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/poles_distance_masks"
)

cable_filepaths = list(filter(lambda path: int(path.stem) in timestamps, labels_folder.glob("*.png")))
poles_filepaths = list(filter(lambda path: int(path.stem) in timestamps, poles_distance_masks_folder.glob("*.npy")))
assert len(cable_filepaths) == len(poles_filepaths)

In [8]:
from torch import nn
from tqdm import tqdm

from powerlines.data.utils import load_processed_label_image, downsample_labels

downsampler = nn.MaxPool2d(kernel_size=2, stride=2).cuda()

frame_size = 187 * 256  # downsampled 16x shape
total_pos_count = 0

In [14]:
# Cable frequency
total_cable_px = 0
for filepath in tqdm(cable_filepaths):
    label = load_processed_label_image(filepath)
    total_cable_px = np.count_nonzero(label)
    downsampled_label = downsample_labels(
        torch.from_numpy(label).unsqueeze(0).float(), grid_size=16, adjust_to_divisible=False
    ).long().detach().cpu().numpy()

    pos_count = np.count_nonzero(downsampled_label == 1)
    total_pos_count += pos_count

pos_frequency = total_pos_count / (len(cable_filepaths) * frame_size)
neg_frequency = 1 - pos_frequency

print(f"Cables frequency: {pos_frequency}")
print(f"Cables weight: {1 / pos_frequency}")
print(f"Background weight: {1 / neg_frequency}")

100%|██████████| 865/865 [02:16<00:00,  6.35it/s]


In [15]:
from powerlines.data.utils import nevbw_labels_from_distance_mask

# Poles frequency
total_poles_px = 0
for filepath in tqdm(poles_filepaths):
    poles_distance_mask = np.load(filepath)
    total_poles_px += np.count_nonzero(poles_distance_mask == 0)
    poles_label = nevbw_labels_from_distance_mask(poles_distance_mask)
    downsampled_poles_label = downsample_labels(
        torch.from_numpy(poles_label).float(), grid_size=16, adjust_to_divisible=False
    ).long().detach().cpu().numpy()

    pos_count = np.count_nonzero(downsampled_poles_label == 1)
    total_pos_count += pos_count

pos_frequency = total_pos_count / (len(poles_filepaths) * frame_size)
neg_frequency = 1 - pos_frequency

print(f"Poles frequency: {pos_frequency}")
print(f"Poles weight: {1 / pos_frequency}")
print(f"Background weight: {1 / neg_frequency}")

100%|██████████| 865/865 [00:12<00:00, 66.61it/s]
